In [ ]:
!pip install --upgrade --force-reinstall "transformers[torch]" --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [ ]:
!pip install "accelerate"

In [ ]:
!pip install "numpy<2" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contr

In [ ]:
#After Restart session
import langchain
import langchain_core
import langchain_openai
import langchain_community
from importlib.metadata import version
import transformers
import numpy as np

print(version("langchain"))
print(version("langchain-core"))
print(version("langchain-openai"))
print(version("langchain-community"))

print("langchain_v",langchain.__version__)
print("langchain_core",langchain_core.__version__)
print("langchain_community",langchain_community.__version__)
print("transformers",transformers.__version__)
print("numpy",np.__version__)


1.2.3
1.2.7
1.1.7
0.4.1
langchain_v 1.2.3
langchain_core 1.2.7
langchain_community 0.4.1
transformers 4.57.3
numpy 1.26.4


In [ ]:
!pip install -U langchain-huggingface

In [ ]:

from langchain_huggingface import HuggingFacePipeline
from langchain_community.agent_toolkits.load_tools import load_tools as lc_load_tools
from transformers import pipeline
from langchain_core.runnables import RunnableSequence

In [ ]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256,
    temperature=0.0,
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=c433525ea240168c610a75a2802d7a82a72334ad13fdcc1ec09ed5e136fbf98d
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [ ]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)
tools = lc_load_tools(["llm-math", "wikipedia"], llm=llm)

In [ ]:
for t in tools:
    print(t.name, "->", t.description)

Calculator -> Useful for when you need to answer questions about math.
wikipedia -> A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.


In [ ]:
def get_tool_by_keyword(keyword):
    for t in tools:
        if keyword.lower() in t.name.lower():
            return t
    return None


def agent(query: str):
    q = query.lower()

    # Math routing
    if any(x in q for x in ["*", "+", "-", "/", "calculate", "math"]):
        tool = get_tool_by_keyword("math") or get_tool_by_keyword("calc")
        if tool:
            return tool.run(query)

    # Wikipedia routing
    if any(x in q for x in ["who", "what", "where", "capital", "wiki"]):
        tool = get_tool_by_keyword("wiki")
        if tool:
            return tool.run(query)

    # Fallback
    return llm.invoke(query)

In [ ]:
agent("What is the capital of France?")
agent("Explain quantum computing")

'Quantum computing (QCC) is a technique that combines the properties of a quantum wave with the properties of a non-resonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum 

In [ ]:
#Using a bigger Model
hf_pipeline = pipeline(
    "text-generation",
    model="tiiuae/falcon-7b-instruct",
    device=0,
    max_new_tokens=256,
    temperature=0.0,
)
llm_model = HuggingFacePipeline(pipeline=hf_pipeline)

tools = lc_load_tools(["llm-math", "wikipedia"], llm=llm_model)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# ReAct prompt
prompt = hub.pull("hwchase17/react")

# Agent
agent = create_react_agent(llm=llm_model, tools=tools, prompt=prompt)

# Executor
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

#If throws an error then Upgrade all your LangChain packages to a modern, compatible version that includes create_react_agent


'from langchain.agents import create_react_agent, AgentExecutor\nfrom langchain import hub\n\n# ReAct prompt\nprompt = hub.pull("hwchase17/react")\n\n# Agent\nagent = create_react_agent(llm=llm_model, tools=tools, prompt=prompt)\n\n# Executor\nexecutor = AgentExecutor(\n    agent=agent,\n    tools=tools,\n    verbose=True,\n    handle_parsing_errors=True\n)\n\n#If throws an error then Upgrade all your LangChain packages to a modern, compatible version that includes create_react_agent\n'

In [ ]:
question = "What is the capital of France and what is 17*23?"
result = executor.invoke({"input": question})
print(result["output"])

In [ ]:
def agent_router(query):
    if any(op in query for op in ["+", "-", "*", "/", "calculate"]):
        tool = next(t for t in tools if "Calculator" in t.name)
        return tool.run(query)
    elif any(k in query.lower() for k in ["who", "what", "where", "capital", "wiki"]):
        tool = next(t for t in tools if "Wikipedia" in t.name)
        return tool.run(query)
    else:
        return llm.invoke(query)

print(agent_router(question))

In [ ]:
#Upgrading & working with GPT model
!pip install -U \
  langchain==0.3.14 \
  langchain-core==0.3.32 \
  langchain-community==0.3.14 \
  langchain-openai==0.2.14 \
  pydantic==2.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 17.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.2
    Uninstalling pydantic_core-2.33.

In [ ]:
#After restarting session
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.agent_toolkits.load_tools import load_tools
import os

In [ ]:
from dotenv import load_dotenv
load_dotenv("/content/.env")

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
    deployment_name="gpt-4.1",
    api_version="2024-12-01-preview",
    temperature=0,
)



In [ ]:
from langchain_community.agent_toolkits.load_tools import load_tools as lc_load_tools
tools = lc_load_tools(["llm-math", "wikipedia"], llm=llm)
from langchain import hub
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

executor.invoke(
    {"input": "What is the capital of France and what is 17 * 23?"}
)

/usr/local/lib/python3.12/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
The question has two parts: the capital of France and a multiplication problem. First, I need to find the capital of France, then calculate 17 * 23.
Action: wikipedia
Action Input: capital of FrancePage: List of capitals of France
Summary: This is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944.

Page: Capital punishment in France
Summary: Capital punishment in France (French: peine de mort en France) is banned by Article 66-1 of the Constitution of the French Republic, voted as a constitutional amendment by the Congress of the French Parliament on 19 February 2007 and simply stating "No one can be sentenced to the death penalty" (French: Nul ne peut être condamné à la peine de mort). The death penalty was already declared illegal on 9 October 1981 when President François Mitterrand signed a law prohibiting the judicial system from using it and commuting the sentences of the seven peop

{'input': 'What is the capital of France and what is 17 * 23?',
 'output': 'The capital of France is Paris, and 17 * 23 equals 391.'}

In [ ]:
question = "Alan Turing was a British mathematician and computer scientist, what is he famous for?"
result = executor.invoke({"input": question})
print(result)



> Entering new AgentExecutor chain...
Thought: I should look up information about Alan Turing to provide a comprehensive answer about what he is famous for.
Action: wikipedia
Action Input: Alan TuringPage: Alan Turing
Summary: Alan Mathison Turing (; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer science.
Born in London, Turing was raised in southern England. He graduated from King's College, Cambridge, and in 1938, earned a doctorate degree from Princeton University. During World War II, Turing worked for the Government Code and Cypher School at Bletchley Park, Britain's codebreaking centre